In [ ]:
from fooof import FOOOF
import numpy as np
from sklearn.utils import resample

# for plotting, remove later
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.image as mpimg

import scipy.stats as st

from numpy import std, mean, sqrt

def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)

def getCI(sample,n_boot,alpha):
    
    '''
    Takes a 2D array and boostrapps each column. Therefore, structure 2D array as 
    [[sample1],[sample2],...] to boostrap each data point.
    '''
    
    lower_p = (alpha/2.0) * 100
    upper_p = (1-(alpha/2.0)) * 100
    sample = np.asarray(sample)
            
    # bootstrap each column (ie data point)
    transposed_sample = np.transpose(sample)
    bootmeans = []
    for row in transposed_sample:
        rowmean = []
        for i in range(n_boot):
            resamplesize = round(row.size*.8)
            samples = resample(row,replace=True,n_samples=resamplesize)
            rowmean.append(samples.mean())
        bootmeans.append(rowmean)

    #mean of each row's bootstrapped samples
    means = []
    for row in bootmeans:
        means.append(np.mean(row))

    #get CI for each row
    CIs_lower,CIs_upper = [],[]

    for row in bootmeans:
        CIs_lower.append(np.percentile(row,lower_p))
        CIs_upper.append(np.percentile(row,upper_p))

    return means,CIs_lower,CIs_upper


EEG_h = np.load('/home/e/etayhay/frankm/Mazza2023/data/simulations_processed/EEG_h.npy',allow_pickle=True).item(0)
EEG_m = np.load('/home/e/etayhay/frankm/Mazza2023/data/simulations_processed/EEG_m.npy',allow_pickle=True).item(0)

freqs = EEG_h['mean_psd']['freq_wel']

EEGs_h = np.array([EEG_h[str(i+1000)]['ps_wel'] for i in range(60)])
EEGs_m = np.array([EEG_m[str(i+2000)]['ps_wel'] for i in range(60)])

In [ ]:
# low,high = 12,91
EEGs_comb = np.array([EEGs_h,EEGs_m])

aps = np.zeros((2,60,82))
ps = np.zeros((2,60,82))

total_fits = np.zeros((2,60,82))

peak_width_limits=[2,6] 
max_n_peaks= 3.
min_peak_height = 0 #.5
peak_threshold = 2 
aperiodic_mode='fixed'
verbose=True

n_boot = 100


exponents = [[[] for _ in range(60)] for _ in range(2)]
offsets = [[[] for _ in range(60)] for _ in range(2)]
peak_params = [[[] for _ in range(60)] for _ in range(2)]


for c in range(2):         # healthy, MDD
    for r in range(60):    # seed number
        aps_temp = []
        ps_temp = []
        
        # fit fooof to psd
        single_fooof = FOOOF(peak_width_limits=peak_width_limits,
                              max_n_peaks=max_n_peaks,
                              min_peak_height=min_peak_height,
                              peak_threshold=peak_threshold,
                              aperiodic_mode=aperiodic_mode,
                              verbose=verbose)
        
        single_fooof.fit(freqs,EEGs_comb[c][r],freq_range=[3,30]) #old was 2,30
        
        # fits are log-values, transform accordingly
        aps_temp = 10**single_fooof._ap_fit # aperiodic fit
        ps_temp = single_fooof._peak_fit    # periodic fit
        
        # params
        offsets[c][r] = single_fooof.aperiodic_params_[0]
        exponents[c][r] = single_fooof.aperiodic_params_[1]
        peak_params[c][r] = single_fooof.peak_params_
        
        # save to array so I can average / bootstrap after
        aps[c][r] = aps_temp  
        ps[c][r] = ps_temp
        
        total_fits[c][r]=10**single_fooof.fooofed_spectrum_

#         fig  = plt.figure()
#         ax = plt.subplot(111)
#         title = str(c) + ' ' + str(r)
#         ax.set_title(title)
#         single_fooof.plot(ax = ax)

In [ ]:
EEG_bs = [[0,0,0],[0,0,0]]
aps_bs = [[0,0,0],[0,0,0]]
ps_bs = [[0,0,0],[0,0,0]]

total_fits = np.array(total_fits)
total_fits_bs = [[0,0,0],[0,0,0]]

for c in range(2):
    EEG_bs[c][0],EEG_bs[c][1],EEG_bs[c][2] = getCI(EEGs_comb[c,:],n_boot=100,alpha=.05)
    aps_bs[c][0],aps_bs[c][1],aps_bs[c][2] = getCI(aps[c,:],n_boot=100,alpha=.05)
    ps_bs[c][0],ps_bs[c][1],ps_bs[c][2] = getCI(ps[c,:],n_boot=100,alpha=.05)
    total_fits_bs[c][0],total_fits_bs[c][1],total_fits_bs[c][2] = getCI(total_fits[c,:],n_boot=100,alpha=.05)

EEG_bs = np.array(EEG_bs)
aps_bs = np.array(aps_bs)
ps_bs = np.array(ps_bs)
total_fits_bs = np.array(total_fits_bs)

## Traditional Band Changes

In [ ]:

# 5 - 15Hz

lfreq,hfreq = 5,15

low = np.where(freqs==lfreq)[0][0]
high = np.where(freqs==hfreq)[0][0]

EEG_h['mean_psd']['ps_wel']


diff = (EEG_m['mean_psd']['ps_wel'][low:high]-EEG_h['mean_psd']['ps_wel'][low:high]) / EEG_h['mean_psd']['ps_wel'][low:high]
diff_percent = round(diff.mean()*100,2)
print(f'From {lfreq} - {hfreq} Hz, MDD showed a {diff_percent}% increase')

ps = []
for f in range(low,high):
    _,p = st.ttest_ind(EEGs_h[:,f],EEGs_m[:,f])
    ps.append(p)

print(f'Mean p-val: {np.mean(ps)}')    
    
# significance
ds = []
for f in range(low,high):
    ds.append(cohen_d(EEGs_m[:,f],EEGs_h[:,f]))
    
    
print(f'Mean d: {round(mean(ds),2)} +/- {round(std(ds),2)}')

## Aperiodic

In [ ]:
# Aperiodic Parameters

pvals_ap = [None, None]

unlog_off_h = 10**np.array(offsets[0])
unlog_off_m = 10**np.array(offsets[1])  
pvals_ap[0] = round(st.ttest_ind(unlog_off_h,unlog_off_m)[1],5)

print('Offsets')
print('H off:', unlog_off_h.mean(), '+-', unlog_off_h.std())
print('M off:', unlog_off_m.mean(), '+-', unlog_off_m.std())
print('H vs MDD:',unlog_off_h.mean(), '\t ', unlog_off_m.mean(),'\t % change=',round(((unlog_off_m.mean() - unlog_off_h.mean()) / unlog_off_h.mean())*100,2),'\t p=', round(st.ttest_ind(unlog_off_h,unlog_off_m)[1],5), '\t d=',round(cohen_d(unlog_off_m, unlog_off_h),2))


exp_h = np.array(exponents[0])
exp_m = np.array(exponents[1])
pvals_ap[1] = round(st.ttest_ind(exp_h,exp_m)[1],5)

print('\n')
print('Exponents')

print('H exp:', exp_h.mean(), '+-', exp_h.std())
print('M exp:', exp_m.mean(), '+-', exp_m.std())
print('H vs MDD:',exp_h.mean(), '\t ', exp_m.mean(),'\t % change=',round(((exp_m.mean() - exp_h.mean()) / exp_h.mean())*100,2),'\t p=', round(st.ttest_ind(exp_h,exp_m)[1],5), '\t d=',round(cohen_d(exp_m, exp_h),2))




fig,axes = plt.subplots(figsize=(10,5), ncols=2, nrows=1)
colors = ['k','r']

axes[0].violinplot(10**np.array(offsets[0]),[0], showmeans=True, showextrema=False)
axes[0].violinplot(10**np.array(offsets[1]),[1], showmeans=True, showextrema=False)
axes[0].annotate(xy=(0.2,np.mean(axes[0].get_ylim())), s='p= '+str(pvals_ap[0]))

axes[1].violinplot(exponents[0],[0], showmeans=True, showextrema=False)
axes[1].violinplot(exponents[1],[1], showmeans=True, showextrema=False)
axes[1].annotate(xy=(0.2,np.mean(axes[1].get_ylim())), s='p= '+str(pvals_ap[1]))

[[ax.spines[spine].set_visible(False) for spine in ['top','right']] for ax in axes]
        
[ax.set_xlim(-1,2) for ax in axes]
[ax.set_xticks([0,1]) for ax in axes]
[ax.set_xticklabels(['HC','MDD']) for ax in axes]
axes[0].set_title('Offsets')
axes[1].set_title('Exponents')


fig,axes = plt.subplots(figsize=(10,5),ncols=2)

colors = ['tab:blue','tab:orange']
for ax in range(2):
    for c in range(2):
        axes[ax].plot(single_fooof.freqs,aps_bs[c][0],color=colors[c])
        axes[ax].fill_between(single_fooof.freqs,aps_bs[c][1],aps_bs[c][2],color=colors[c], alpha=.2)
        
[[ax.spines[spine].set_visible(False) for spine in ['top','right']] for ax in axes]
axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
axes[1].get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
_ = axes[1].set_xticks([4,10,30])



## Aperiodic power between 4 - 30Hz


In [ ]:
l = np.where(single_fooof.freqs == 4)[0][0]
h = np.where(single_fooof.freqs == 30)[0][0]

percent_change = np.mean((aps[1,:][l:h].mean(axis=0) - aps[0,:][l:h].mean(axis=0)) / aps[0,:][l:h].mean(axis=0))*100
print('% change:',percent_change)

ps = []
ds = []

for f in range(l,h):
    _,p = st.ttest_ind(aps[1,:,f],aps[0,:,f])
    ps.append(p)
    
    ds.append(cohen_d(aps[1,:,f],aps[0,:,f]))
    
print('p',np.mean(ps))
print('d',np.mean(ds))


## Periodic

In [ ]:
pvals = [[None, None, None],[None, None, None],[None, None, None]]


conds = ['HC','MDD']

bandlims = [[4,8],[8,13],[13,16]]

for b, band in enumerate(bandlims):
    # 0 = cf, 1 = power, 2 = bw
    print('\n')
    print(band)
    
    h_cf = [peak[0] for peak in np.concatenate(peak_params[0]) if band[0]<= peak[0] <= band[1]]
    m_cf = [peak[0] for peak in np.concatenate(peak_params[1]) if band[0]<= peak[0] <= band[1]]
    
    h_p = [peak[1] for peak in np.concatenate(peak_params[0]) if band[0]<= peak[0] <= band[1]]
    m_p = [peak[1] for peak in np.concatenate(peak_params[1]) if band[0]<= peak[0] <= band[1]]
    
    h_bw = [peak[2] for peak in np.concatenate(peak_params[0]) if band[0]<= peak[0] <= band[1]]
    m_bw = [peak[2] for peak in np.concatenate(peak_params[1]) if band[0]<= peak[0] <= band[1]]
    
    print('n peaks H vs MDD:\t', len(h_cf),'\t\t', len(m_cf))
    print('Center Freq H vs MDD:\t', round(np.mean(h_cf),2), '+-', round(np.std(h_cf),2),'\t',round(np.mean(m_cf),2), '+-', round(np.std(m_cf),2), '\t % change=',round(((np.mean(m_cf) - np.mean(h_cf)) / np.mean(h_cf))*100,2), '\t p=',round(st.ttest_ind(h_cf,m_cf)[1],5), '\t d=', round(cohen_d(m_cf,h_cf),2))
    print('Peak Power H vs MDD:\t', round(np.mean(h_p),2), '+-', round(np.std(h_p),2),'\t',round(np.mean(m_p),2), '+-', round(np.std(m_p),2), '\t % change=',round(((np.mean(m_p) - np.mean(h_p)) / np.mean(h_p))*100,2), '\t p=',round(st.ttest_ind(h_p,m_p)[1],5), '\t d=', round(cohen_d(m_p,h_p),2))
    print('Bandwidth H vs MDD:\t', round(np.mean(h_bw),2), '+-', round(np.std(h_bw),2),'\t',round(np.mean(m_bw),2), '+-', round(np.std(m_bw),2), '\t % change=',round(((np.mean(m_bw) - np.mean(h_bw)) / np.mean(h_bw))*100,2), '\t p=',round(st.ttest_ind(h_bw,m_bw)[1],5), '\t d=', round(cohen_d(m_bw,h_bw),2))
    
    pvals[b][0] = round(st.ttest_ind(h_cf,m_cf)[1],5)
    pvals[b][1]= round(st.ttest_ind(h_p,m_p)[1],5)
    pvals[b][2]= round(st.ttest_ind(h_bw,m_bw)[1],5)
    
fig, axes = plt.subplots(figsize=(10,10), ncols=3, nrows=3, sharex=True)

for b, band in enumerate(bandlims):

    
    h_cf = [peak[0] for peak in np.concatenate(peak_params[0]) if band[0]<= peak[0] <= band[1]]
    m_cf = [peak[0] for peak in np.concatenate(peak_params[1]) if band[0]<= peak[0] <= band[1]]
    
    h_p = [peak[1] for peak in np.concatenate(peak_params[0]) if band[0]<= peak[0] <= band[1]]
    m_p = [peak[1] for peak in np.concatenate(peak_params[1]) if band[0]<= peak[0] <= band[1]]
    
    h_bw = [peak[2] for peak in np.concatenate(peak_params[0]) if band[0]<= peak[0] <= band[1]]
    m_bw = [peak[2] for peak in np.concatenate(peak_params[1]) if band[0]<= peak[0] <= band[1]]
    
    for p,param in enumerate([[h_cf, m_cf],[h_p,m_p],[h_bw,m_bw]]):
        axes[p][b].violinplot(dataset = param[0], positions = [0], showmeans=True, showextrema=False)
        axes[p][b].violinplot(dataset = param[1], positions = [1], showmeans=True, showextrema=False)
        
        axes[p][b].set_xticks([0,1])
        axes[p][b].set_xticklabels(['HC','MDD'])
        [axes[p][b].spines[spine].set_visible(False) for spine in ['top','right']]
        
        if pvals[b][p]<0.05:
            axes[p][b].annotate(xy=(0.2,np.mean(axes[p][b].get_ylim())), s='p= '+str(pvals[b][p]))
            
axes[0][0].set_ylabel('Center Frequency (Hz)')
axes[1][0].set_ylabel('Power (a.u.)')
axes[2][0].set_ylabel('Bandwidth (Hz)')

axes[0][0].set_title('Theta 4 - 8')
axes[0][1].set_title('Alpha 8 - 12')
axes[0][2].set_title('Beta 12 - 16')

fig,axes = plt.subplots(figsize=(7,7))

for c in range(2):    
    axes.plot(single_fooof.freqs, ps_bs[c][0],color=colors[c])
    axes.fill_between(single_fooof.freqs, ps_bs[c][1], ps_bs[c][2], color=colors[c], alpha=.1)


In [ ]:
path = '/home/e/etayhay/frankm/Mazza2023/data/figures/Figure3/'

np.save(path + 'single_fooof_freqs.npy',single_fooof.freqs)

np.save(path + 'EEG_bs.npy',EEG_bs)
np.save(path + 'aps_bs.npy',aps_bs)
np.save(path + 'ps_bs.npy',ps_bs)